# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [10]:
# Dependencies and Setup
import warnings
warnings.filterwarnings("ignore")
import hvplot.pandas
import pandas as pd
import holoviews
import geoviews
import requests
import json
from bokeh.models import HoverTool

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,dingle,10.9995,122.6711,22.87,93,54,3.43,PH,1676223433
1,1,ancud,-41.8697,-73.8203,15.40,59,100,3.76,CL,1676223228
2,2,lebedinyy,58.5167,125.5167,-16.70,97,100,7.05,RU,1676223433
3,3,san andres,12.5847,-81.7006,29.01,70,40,7.20,CO,1676223433
4,4,albany,42.6001,-73.9662,7.91,44,99,0.07,US,1676223236


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [14]:
plot_city_map = city_data_df.hvplot.points(
    x="Lng", y="Lat", size="Humidity", geo = True, tiles = "EsriNatGeo", color = "City")

# Plot the city map using the function
plot_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values

# Narrow down the city_data_df DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 27 degrees but higher than 21
# Wind speed less than 4.5 m/s
# Zero cloudiness

# print(len(city_data_df))
city_data_df.dropna(inplace=True)
# print(len(city_data_df))

Ideal_Cities_df = city_data_df[(city_data_df['Wind Speed'] < 4.5) & 
                               (city_data_df['Cloudiness'] == 0) & 
                               (city_data_df['Max Temp'] > 21) &
                               (city_data_df['Max Temp'] < 27)]
print(len(Ideal_Cities_df))
Ideal_Cities_df.head()

10


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
181,181,salalah,17.0151,54.0924,24.05,64,0,1.03,OM,1676223498
183,183,thoen,17.6129,99.2161,21.43,38,0,1.15,TH,1676223499
216,216,keti bandar,24.1447,67.4497,23.37,27,0,3.45,PK,1676223511
314,314,chimbarongo,-34.7000,-71.0500,26.49,16,0,2.78,CL,1676223549
376,376,chabahar,25.2919,60.6430,21.72,61,0,1.27,IR,1676223572


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = Ideal_Cities_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
181,salalah,OM,17.0151,54.0924,64,
183,thoen,TH,17.6129,99.2161,38,
216,keti bandar,PK,24.1447,67.4497,27,
314,chimbarongo,CL,-34.7000,-71.0500,16,
376,chabahar,IR,25.2919,60.6430,61,
422,catuday,PH,16.2923,119.8062,76,
429,alice springs,AU,-23.7000,133.8833,46,
438,goure,NE,13.9835,10.2704,15,
453,sitrah,BH,26.1547,50.6206,71,
552,emiliano zapata,MX,18.8333,-99.1833,35,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories": "accommodation.hotel",
    "limit": 1,
    "apiKey": geoapify_key
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{lng},{lat},{radius}"
    params["bias"] = f"proximity:{lng},{lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    hotel_response = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = hotel_response.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
salalah - nearest hotel: Muscat International Hotel
thoen - nearest hotel: No hotel found
keti bandar - nearest hotel: No hotel found
chimbarongo - nearest hotel: Hotel Central
chabahar - nearest hotel: هتل سپیده
catuday - nearest hotel: Nina Anchors Inn
alice springs - nearest hotel: Aurora Alice Springs
goure - nearest hotel: No hotel found
sitrah - nearest hotel: No hotel found
emiliano zapata - nearest hotel: Radisson Hotel Quinta Rubelinas Cuernavaca


,City,Country,Lat,Lng,Humidity,Hotel Name
181,salalah,OM,17.0151,54.0924,64,Muscat International Hotel
183,thoen,TH,17.6129,99.2161,38,No hotel found
216,keti bandar,PK,24.1447,67.4497,27,No hotel found
314,chimbarongo,CL,-34.7000,-71.0500,16,Hotel Central
376,chabahar,IR,25.2919,60.6430,61,هتل سپیده
422,catuday,PH,16.2923,119.8062,76,Nina Anchors Inn
429,alice springs,AU,-23.7000,133.8833,46,Aurora Alice Springs
438,goure,NE,13.9835,10.2704,15,No hotel found
453,sitrah,BH,26.1547,50.6206,71,No hotel found
552,emiliano zapata,MX,18.8333,-99.1833,35,Radisson Hotel Quinta Rubelinas Cuernavaca


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [15]:
# Configure the map plot
def plot_hotel_map(hotel_df):
    # Create a hover tool with the desired columns
    hover = HoverTool(
        tooltips=[
            ("City", "@City"),
            ("Country", "@Country"),
            ("Hotel Name", "@Hotel Name")
        ]
    )

    # Add the hover tool to the plot
    # Display the map
    plot = hotel_df.hvplot.points(x="Lng", y="Lat", size="Humidity", geo = True, tiles = "EsriNatGeo", color = "City", hover_cols=["City", "Country", "Hotel Name"]).opts(tools=[hover])
    return plot

plot_hotel_map(hotel_df)

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)